This is a simple implementation of steganography
Hiding text inside the image

In [ ]:
def generate_binary_8bitdata(data):
    # make a list of binary codes of given data
    bin_data = []
    for i in data:
      # format represents the how the value will be represented as 
      # ord Return the Unicode code point for a one-character string,
      # '08b' convert int to binary from ASCII code of text
        bin_data.append(format(ord(i), '08b'))
    return bin_data

In [ ]:
# PIL enables to use image in python
# without this libracy we can not import and export image inside our file
from PIL import Image

In [ ]:
# in pixels we can write/modify according to the 8-bit binary data
def write_in_pixel(pixel, msg):
    data_list = generate_binary_8bitdata(msg)
    # len() finds the lenth of the list here
    bin_length = len(data_list)
    # iter() returs itirator object
    pix_data = iter(pixel)
    for x in range (bin_length):
        # binary data from pixel manupulation (extraction)
        pixel = [
                value for value in 
               (pix_data.__next__ ()[:3] +
                pix_data.__next__ ()[:3] +
                pix_data.__next__ ()[:3])
               ]

        # let's change the pixel value >> odd_pix = 1 and even_pix = 0
        #Digital image constructed by 2-Dimensions array of the pixels. 
        # Each pixel is identified by its coordinates (co-x, co-y) and 
        # its value and characterized by matrix size, pixel depth, and resolution.
        # The size of the matrix is determined by the number of columns (cn) 
        # and the number of rows (rn) of the image matrix (cn × rn).
        # Matrix size is may vary because it is selected by the operator. 
        # Higher the matrix dimension higher the resolution of the image.
        for y in range (0, 8):
            if (data_list[x][y]) == '0' and (pixel[y] % 2) != 0:
                pixel[y] = pixel[y] - 1

            elif (data_list[x][y]) == '1' and (pixel[y] % 2) == 0:
                if pixel[y] != 0:
                    pixel[y] = pixel[y] - 1
                else:
                    pixel[y] = pixel[y] + 1  
        # doing negation or subtraction because need to work on the first value of list whose index is 0
        if x == (bin_length - 1):
            if pixel[-1] % 2 == 0:
                if pixel[-1] != 0:
                    pixel[-1] = pixel[-1] - 1
                else:
                    pixel[-1] = pixel[-1] + 1
        else:
            if pixel[-1] % 2 != 0:
                pixel[-1] = pixel[-1] - 1
                
        # yield executes previous yield state also if there is any
        # the following yeild act as a pixel code generator
        pixel = tuple(pixel)
        # we added pixel value before now we can move from one value to another
        yield pixel[0:3]
        yield pixel[3:6]
        yield pixel[6:9]

In [ ]:
# This function will take image and message as input
def encode_enc(image_ref, msg):
    size_img = image_ref.size[0]

    # create a pixel tuple and go through for loop
    (x, y) = (0, 0)
    for pixel in write_in_pixel(image_ref.getdata(), msg):
        # Putting modified pixel tuple in the new image
        image_ref.putpixel((x, y), pixel)
        if x == size_img - 1:
            x = 0
            y = y + 1
        else:
            x = x + 1

In [ ]:
# to remove unnecessary spaces in string data
def remove(string):
  return string.replace(" ", "")

In [ ]:
# Encode data into image
def hide_message():
    input_img = input("\nEnter image name *with extension*:\n")
    while len(input_img) == 0:
      input_img = input("\nEnter image name *with extension*:\n")

    try:
      # opening image in reading mode
      image = Image.open (input_img, 'r')
      # if len(image) == 0:
      #   raise FileNotFoundError("No such image found")
      #   main()
    except:
       print("No such file fund, according to the given name. Try again.\n")
       main() 
    finally:    
      message = input ("\nEnter you message to hide :\n")
      while len(message) == 0:
        print("No message given. Try again.")
        message = input ("\nEnter you message to hide\n")

      image_ref = image.copy()
      # let's load the image and message to the funtion
      encode_enc(image_ref, message)

      # for better practice let's create new image with hidden message
      new_img_name = input ("\nSystem will create new image.\n Enter name *without extension*:\n")
      if len(new_img_name) == 0:
        new_img_name = input("\nSystem will create new image.\n Enter name *without extension*:\n")
      else:
        # The following code creats new image ext with png
        image_ref.save(f'{remove(new_img_name)}.png')
        

In [ ]:
# Decode the data in the image
def extract_message():  
    text = ''
    img_name = input ("\nYour image name *with extension* to the system:\n")    
    
    while len(img_name) == 0:
      img_name = input ("\nYour image name *with extension* to the system:\n")

    # opening image in reading mode
    try:
      image = Image.open(img_name, 'r')
      img_data = iter(image.getdata())
    except:
      print("No such file fund, according to the given name. Try again.\n")
      main()
    finally:   
      while True:
          pixels = [pvalue for pvalue in 
                    (img_data.__next__ ()[:3] +
                    img_data.__next__ ()[:3] +
                    img_data.__next__ ()[:3])
                    ]

          # string of binary data
          bin_str_data = ''
          for i in pixels[:8]:
              if (i % 2) == 0:
                  bin_str_data = bin_str_data + '0'
              else:
                  bin_str_data = bin_str_data + '1'

          # storing extracted charecter to the message as a single string
          text = text + chr(int(bin_str_data, 2))
          if (pixels[-1] % 2) != 0:
              return text

In [ ]:
def text_to_file(text):
  # Save the data into text file
  file_steg_data = open("output.txt","w")
  file_steg_data.write(text)
  file_steg_data.close()
  # once it done it creats a .txt file in a working directory

In [ ]:
# Application execution area
def main():
    choice = int(input("Type 1 > To hide message\nType 2 > To extract message\nType 3 > To Exit.\n"))
    print(f'You have selected the option {choice}\n')
    if choice == 1:
      hide_message()
      if hide_message:
        print("Your message has been encoded into the picture.\nThank you for using the system.")
      # else:
      #   main([])
    elif choice == 2:
      hidden_message = extract_message()
      print(f'The message was:\n{hidden_message}\n\nThank you for using the system.')      
      text_to_file(hidden_message)

    elif choice == 3:
      print("Thank you for using the system.")
      exit()
    else:
      print("Choose correct option.")
      main([])

In [ ]:
if __name__ == '__main__':
  main ()